# Join the dataframes

This note-nook counts the number of parks, schools, shopping centers in a suburb. It also aggregates with the proximity to train stations to provide a final csv ready to be fed to ranking system

In [1]:
import pandas as pd 
from pandasql import sqldf


In [8]:
suburb = pd.read_csv("../data/curated/sub_4_pred.csv")
school = pd.read_csv("../data/raw/dv331_schoollocations2022.csv", encoding='cp1252')
parks = pd.read_csv("../data/curated/suburb_w_leisure.csv")
shop = pd.read_csv("../data/raw/ShoppingCentreData.csv")
train = pd.read_csv("../data/curated/suburbs_w_dstTrain.csv")

In [3]:
parks = parks[["Suburb", "Postcode", "NumPlaygrounds", "NumParks"]]
parks.head()

208

In [4]:
school = school[["School_Name", "Postal_Postcode"]]
school.head()

,School_Name,Postal_Postcode
0,Alberton Primary School,3971
1,Allansford and District Primary School,3277
2,Avoca Primary School,3467
3,Avenel Primary School,3664
4,Warrandyte Primary School,3113


In [32]:
train = train[["Suburb", "Postcode", "DstToStation"]]
train.head()

,Suburb,Postcode,DstToStation
0,"St Kilda, St Kilda South, St Kilda West",3182,1444.0
1,"Armadale North, Armadale",3143,307.7
2,"Carlton South, Carlton",3053,1978.0
3,"Melbourne University, Parkville",3052,1139.4
4,"St Kilda, St Kilda South, St Kilda West",3182,1444.0


In [14]:
shop = shop[["Suburb"]]
shop.head()

,Suburb
0,Melbourne
1,St Kilda
2,Melbourne
3,Balnarring
4,Brunswick


In [34]:
# Start off with merging shops and parks 
query = "SELECT * FROM parks LEFT OUTER JOIN shop ON shop.Suburb = parks.Suburb"
result = sqldf(query)
result.head()


,Suburb,Postcode,NumPlaygrounds,NumParks,Suburb
0,Aberfeldie,3040,0,0,None
1,Albanvale,3021,0,0,None
2,Albion,3020,1,0,None
3,Alphington,3078,3,0,None
4,Altona,3018,0,0,None
...,...,...,...,...,...
254,Williamstown,3016,4,0,None
255,Williamstown North,3016,1,0,None
256,Windsor,3181,3,0,None
257,Woori Yallock,3139,1,0,None


In [35]:
# Joining with shopping centers
query  = "SELECT result.Postcode, SUM(NumPlaygrounds) as NumPlaygrounds, SUM(NumParks) as NumParks, COUNT(result.Suburb) as NumShoppingCentres \
    FROM result LEFT OUTER JOIN train ON result.Postcode = train.Postcode \
        GROUP BY result.Postcode"
result2 = sqldf(query)
result2

,Postcode,NumPlaygrounds,NumParks,NumShoppingCentres
0,3000,26,0,26
1,3006,0,0,1
2,3008,1,0,1
3,3011,7,0,2
4,3015,0,2,0
...,...,...,...,...
85,3810,36,0,2
86,3915,2,0,0
87,3931,12,3,3
88,3936,1,0,1


In [36]:
# Join with schools
query  = "SELECT Postcode, NumPlaygrounds, NumParks, NumShoppingCentres, COUNT(School_Name) as NumSchool \
    FROM result2 LEFT OUTER JOIN school ON result2.Postcode = school.Postal_Postcode \
        GROUP BY result2.Postcode"
result3 = sqldf(query)
result3.head()

,Postcode,NumPlaygrounds,NumParks,NumShoppingCentres,NumSchool
0,3000,26,0,26,4
1,3006,0,0,1,2
2,3008,1,0,1,1
3,3011,7,0,2,4
4,3015,0,2,0,5
...,...,...,...,...,...
85,3810,36,0,2,9
86,3915,2,0,0,4
87,3931,12,3,3,7
88,3936,1,0,1,3


In [43]:
# Join with dst to train station
query  = "SELECT train.Suburb, train.Postcode, NumPlaygrounds, NumParks, NumShoppingCentres, NumSchool, train.DstToStation \
    FROM train INNER JOIN result3 ON result3.Postcode = train.Postcode"
final = sqldf(query)
final

,Suburb,Postcode,NumPlaygrounds,NumParks,NumShoppingCentres,NumSchool,DstToStation
0,"St Kilda, St Kilda South, St Kilda West",3182,9,0,3,5,1444.0
1,"Armadale North, Armadale",3143,0,0,0,2,307.7
2,"Carlton South, Carlton",3053,3,0,0,2,1978.0
3,"Melbourne University, Parkville",3052,2,1,0,3,1139.4
4,"St Kilda, St Kilda South, St Kilda West",3182,9,0,3,5,1444.0
...,...,...,...,...,...,...,...
89,"Dromana, Arthurs Seat, Safety Beach",3936,1,0,1,3,21947.4
90,"Karingal Centre, Karingal, Frankston Heights, ...",3199,17,0,3,20,2250.4
91,"Tuerong, Hastings",3915,2,0,0,4,8203.2
92,Mornington,3931,12,3,3,7,13918.4


In [44]:
# Combine the number of paygrounds and parks as the number of outdoor leisure facilities 
sum_col = final["NumParks"] + final["NumPlaygrounds"]
final["NumParks"] = sum_col

In [47]:
final = final.drop(columns="NumPlaygrounds")
final.to_csv("../data/curated/liveability.csv")